## Провести сравнение RNN, LSTM, GRU на датасете отзывов (из предыдущих занятий/материалов)

In [21]:
import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [22]:
df =  pd.read_excel("отзывы за лето.xls")

In [23]:
df_train = df.loc[:4131]
df_val = df.loc[4132:]

In [24]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_val['Content'] = df_val['Content'].apply(preprocess_text)

C:\Users\marchenko-i-u\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\marchenko-i-u\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [25]:
text_corpus_train = df_train['Content'].values
text_corpus_valid = df_val['Content'].values

In [26]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [52]:
# y_train = df_train["Rating"].values
# y_val = df_val["Rating"].values
num_classes = 6
y_train = keras.utils.to_categorical(df_train["Rating"], num_classes)
y_val = keras.utils.to_categorical(df_val["Rating"], num_classes)

In [53]:
y_train

array([[0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)

#### RNN

In [66]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

In [67]:
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
8/8 [==============================] - 1s 131ms/step - loss: 1.6499 - accuracy: 0.4320 - val_loss: 1.4797 - val_accuracy: 0.7029
Epoch 2/10
8/8 [==============================] - 1s 112ms/step - loss: 1.3870 - accuracy: 0.7372 - val_loss: 1.2929 - val_accuracy: 0.7029
Epoch 3/10
8/8 [==============================] - 1s 112ms/step - loss: 1.2227 - accuracy: 0.7456 - val_loss: 1.1291 - val_accuracy: 0.7029
Epoch 4/10
8/8 [==============================] - 1s 114ms/step - loss: 1.1184 - accuracy: 0.7464 - val_loss: 1.0316 - val_accuracy: 0.7029
Epoch 5/10
8/8 [==============================] - 1s 113ms/step - loss: 1.0644 - accuracy: 0.7431 - val_loss: 0.9857 - val_accuracy: 0.7029
Epoch 6/10
8/8 [==============================] - 1s 114ms/step - loss: 0.9937 - accuracy: 0.7437 - val_loss: 0.9091 - val_accuracy: 0.7077
Epoch 7/10
8/8 [==============================] - 1s 125ms/step - loss: 0.9302 - accuracy: 0.7415 - val_loss: 0.8786 - val_accuracy: 0.7150
Epoch 8/10
8/8 [====

In [68]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

33/33 [==============================] - 2s 50ms/step - loss: 0.8371 - accuracy: 0.7164


Test score: 0.837068498134613
Test accuracy: 0.7164034843444824


#### LSTM

In [69]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
8/8 [==============================] - 3s 420ms/step - loss: 1.7761 - accuracy: 0.5584 - val_loss: 1.7360 - val_accuracy: 0.7029
Epoch 2/10
8/8 [==============================] - 3s 371ms/step - loss: 1.6409 - accuracy: 0.7442 - val_loss: 1.4462 - val_accuracy: 0.7029
Epoch 3/10
8/8 [==============================] - 3s 336ms/step - loss: 1.3720 - accuracy: 0.7448 - val_loss: 1.2562 - val_accuracy: 0.7029
Epoch 4/10
8/8 [==============================] - 3s 339ms/step - loss: 1.2111 - accuracy: 0.7461 - val_loss: 1.0908 - val_accuracy: 0.7029
Epoch 5/10
8/8 [==============================] - 3s 355ms/step - loss: 1.1238 - accuracy: 0.7448 - val_loss: 1.0208 - val_accuracy: 0.7029
Epoch 6/10
8/8 [==============================] - 3s 379ms/step - loss: 1.0799 - accuracy: 0.7466 - val_loss: 0.9958 - val_accuracy: 0.7029
Epoch 7/10
8/8 [==============================] - 3s 357ms/step - loss: 1.0846 - accuracy: 0.7469 - val_loss: 0.9885 - val_accuracy: 0.7029
Epoch 8/10
8/8 [====

In [70]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

33/33 [==============================] - 3s 101ms/step - loss: 0.9799 - accuracy: 0.6967


Test score: 0.9799314737319946
Test accuracy: 0.6967386603355408


#### GRU

In [71]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
8/8 [==============================] - 2s 309ms/step - loss: 1.7720 - accuracy: 0.6070 - val_loss: 1.7345 - val_accuracy: 0.7029
Epoch 2/10
8/8 [==============================] - 2s 301ms/step - loss: 1.6822 - accuracy: 0.7469 - val_loss: 1.5899 - val_accuracy: 0.7029
Epoch 3/10
8/8 [==============================] - 2s 298ms/step - loss: 1.4397 - accuracy: 0.7464 - val_loss: 1.1371 - val_accuracy: 0.7029
Epoch 4/10
8/8 [==============================] - 2s 269ms/step - loss: 1.1221 - accuracy: 0.7474 - val_loss: 0.9870 - val_accuracy: 0.7029
Epoch 5/10
8/8 [==============================] - 2s 273ms/step - loss: 1.0664 - accuracy: 0.7474 - val_loss: 0.9388 - val_accuracy: 0.7029
Epoch 6/10
8/8 [==============================] - 2s 306ms/step - loss: 0.9676 - accuracy: 0.7474 - val_loss: 0.9168 - val_accuracy: 0.7029
Epoch 7/10
8/8 [==============================] - 2s 280ms/step - loss: 0.9117 - accuracy: 0.7477 - val_loss: 0.8709 - val_accuracy: 0.7029
Epoch 8/10
8/8 [====

In [72]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

33/33 [==============================] - 3s 76ms/step - loss: 0.8231 - accuracy: 0.6987


Test score: 0.8231433033943176
Test accuracy: 0.6987354159355164
